In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.vehicle'):
    shutil.rmtree('./file/4.vehicle')

os.makedirs('./file/4.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(f'''https://www.1aauto.com/catalog/product/fit?skuId={input_.loc[a, 'Part Code']}&pathName={input_.loc[a, 'Url'].removeprefix('https://www.1aauto.com').replace('/', '%2F')}''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_th = [text.strip() for text in html.xpath('//table[@class="fitTable sortableTable onea-table"]/thead/tr/th/text()')]

                # = = = = = = = = = = = = = = =

                list_td = [[text.strip() for text in tr.xpath('./td/text()')] for tr in html.xpath('//table[@class="fitTable sortableTable onea-table"]/tbody/tr')]

                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))
                
                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame(list_td, columns=list_th)
                df_temp['OEM'] = oem
                df_temp['No'] = input_.loc[a, 'No']
                df_temp['Url'] = input_.loc[a, 'Url']
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(30):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make', 'Model', 'Year'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.vehicle/{file}',
              f'''./file/4.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：50] - 99.07% - crawler_98 > 98106.https://www.1aauto.com/chevrolet-gmc-passenger-side-tail-light-wiring-harness-diy-solutions-lht08159/i/1azwh00106
[剩余数量：0] - [当前时间：07:46:00]

输出ing...

总数量：1

[状态：ok，尝试次数：1] - 100.00% - crawler_99 > 1.https://www.1aauto.com/bed-lights/i/1axaa00194
[剩余数量：0] - [当前时间：07:47:13]

输出ing...



Progress: 100%|████████████████████████████| 90/90 [00:00<00:00, 4777.60it/s]

Done ~
